<a href="https://colab.research.google.com/github/miyagin15/TwitterSearch/blob/main/AI_learniing_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FastTextをインストール

In [ ]:
!wget https://github.com/facebookresearch/fastText/archive/0.2.0.zip
!unzip 0.2.0.zip
%cd fastText-0.2.0
!make

In [ ]:
import pandas as pd
import csv 
from sklearn.model_selection import train_test_split
!pip install git+https://github.com/facebookresearch/fastText.git@v0.9.2

In [ ]:
news_corpora = pd.read_csv('olympic_tweet.csv',quoting=csv.QUOTE_NONE)
train, test = train_test_split(news_corpora, test_size=0.2, random_state=0)

In [39]:
import csv
with open("./olympic_tweet.csv", "r", errors="", newline="" ) as f:
    lst = csv.reader(f, delimiter=",")
    df = pd.DataFrame(lst)
#df[1:]
df=df[1:].set_axis(['label', 'date', 'text'], axis=1)
df

,label,date,text
1,1,2021/12/07 12:53,協力的な態度を取ってくれたかー。表向きだけじゃないのー？東京オリンピック反対とか批判してた人...
2,1,2021/12/07 12:51,2月オリンピックだからまた緊急事態宣言でそうじゃない？
3,1,2021/12/07 12:54,恵俊彰「そもそもオリンピックと外交は結びつけちゃいけない」なんで？そんなプロトコルあったっけ...
4,1,2021/11/24 14:00,【 SAP、 Salesforce などのアプリに蓄えたデータ資産を最大活用！】さまざまなア...
5,1,2021/12/07 12:54,恵さん「そもそもオリンピックは政治と結びついちゃいけない」東京五輪と政権批判ゴッチャにして騒...
...,...,...,...
2059,0,2010/12/05 0:16,驚いた。北京オリンピックて2008年だったのか(°Д°)
2060,0,2010/12/05 1:29,大平さん再発言：アジア大会の後の負債で秋葉市長自身が苦しんできたと言っていたのに、さらに規模...
2061,0,2010/12/05 0:09,フィギュアスケートのシュルタイスがルトコフコーチと決別した、と聞きました。この二人を前から知...
2062,0,2010/12/05 0:36,こういうのはTASさんが強そうなので、リストに入れてみました。 【マイリスト】TASさんがち...


In [110]:
train, test = train_test_split(df, test_size=0.2, random_state=0)

In [111]:
# print(df['label'].str.cat(df['text'], sep=' '))
# print("__label__"+df['label'] +" "+ df['text'])
# fast_data=("__label__"+df['label'] +" "+ df['text'])
train=train.replace('"', ' ')
train=train.replace('"', ' ')
fast_train_df=("__label__"+train['label'] +" "+ train['text'])
fast_train_df.to_csv("fast_train.txt",index=False,header=False)

test=test.replace('"', ' ')
test=test.replace('"', ' ')
fast_test_df=("__label__"+test['label'] +" "+ test['text'])
fast_test_df.to_csv("fast_test.txt",index=False,header=False)

In [116]:
import fasttext
model = fasttext.train_supervised(input="fast_train.txt",lr=0.5, epoch=500, minn=3, maxn=5,
                                  wordNgrams=3, loss='ova', dim=300, bucket=200000)
model.save_model("news_corpora.bin")

In [117]:
print("ValidData:", model.test('fast_test.txt'))

ValidData: (405, 0.8864197530864197, 0.8864197530864197)


In [177]:
ret = model.predict("今まで頑張って練習した。東京オリンピック頑張る")
print(ret) 

ret1 = model.predict("冬のバンクーバで結果出す")
print(ret1)

(('__label__1',), array([0.99930036]))
(('__label__0',), array([0.92631376]))


In [139]:
text_2010_df=pd.read_csv("olympic_test.csv")


In [182]:
print(model.predict("スケート楽しい"))

(('__label__1',), array([1.00001001]))


In [155]:
text_2010_df=pd.read_csv("olympic_test.csv")
for date, text in zip(text_2010_df['date'], text_2010_df['text']):
    results=model.predict(text)
    print(date, text)
    print(results[0],"の確率: ",round(results[1][0]*100),"%")
    print("-"*50)

2010/12/04 23:21 まさかのオリンピック金メダリストと飲み。
('__label__0',) の確率:  66 %
--------------------------------------------------
2010/12/04 23:46 今日はレスリング部飲み会でした。チャンピオンになった漆谷もオリンピックにいった笹本も今後の人生にエールをおくりたい！
('__label__1',) の確率:  59 %
--------------------------------------------------
2010/12/04 23:14 柔道見学いってきた。強くなりそうな子だと言われた。オリンピックに連れてってもらえるかもしれんな･･
('__label__0',) の確率:  52 %
--------------------------------------------------
2010/12/04 23:44 ライサイドな毎日。 Ly-side Diary: 気づいたらもうすぐファイナルですね。 ：男子シングルもいいな～って思うのはあるんだけどオリ… http://bit.ly/fim1lS #followmeJP  #followdaibosyu
('__label__0',) の確率:  100 %
--------------------------------------------------
2010/12/04 23:24 シドニーの町には他のオーストラリアの町に比して東洋人の多さが目立つのは住みついたという人がいたけどオリンピック建設に大量の中国人が移住してきて
('__label__1',) の確率:  59 %
--------------------------------------------------
2010/12/04 23:19 「坂の上の雲」の広瀬中佐役の藤本さんは元オリンピック選手です http://bit.ly/hCM9Yb
('__label__0',) の確率:  100 %
--------------------------------------------------
2010/12/04 23:18 そういえば石原慎太郎様いまだに何の責任もとらずオリンピックを招致できな